In [ ]:
from irrep.bandstructure import BandStructure

import numpy as np
import scipy
from wannierberri.symmetry.projections import Projection, ProjectionsSet
from wannierberri.w90files import Wannier90data

bands_pw = np.loadtxt("../../data/diamond/diamond.bands_pw.dat.gnu")
bohr_ang = scipy.constants.physical_constants['Bohr radius'][0] / 1e-10
alatt = 6.1* bohr_ang
bands_pw[:,0]*= 2*np.pi/alatt



bandstructure = BandStructure(code="espresso", 
                              prefix="../../data/diamond/di-irred", 
                              normalize=False,
                              irreducible=True)
sg = bandstructure.spacegroup
# sg.show()

positions_bonds = [[ 0,0,0], [0,0,1/2], [0,1/2,0], [1/2,0,0] ]
proj_bond = Projection(position_num=positions_bonds,  orbital="s", spacegroup=sg)

positions_atoms = [[1/8,1/8,1/8], [-1/8,-1/8,-1/8]] 
proj_s = Projection(position_num=positions_atoms, orbital="s", spacegroup=sg)
proj_p = Projection(position_num=positions_atoms, orbital="p", spacegroup=sg)
# proj_sp3 = Projection(position_num=positions_atoms, orbital="sp3", rotate_basis=False, spacegroup=sg)
projections = ProjectionsSet([proj_s, proj_p])

projections = ProjectionsSet([proj_bond])


w90data = Wannier90data().from_bandstructure(bandstructure, 
                                             projections=projections, 
                                             files=["mmn", "amn", "eig", "symmetrizer"], 
                                             normalize=False,
                                             irreducible=True,
                                             read_npz_list=[],
                                             write_npz_list=[])

# w90data.get_file("symmetrizer").data.keys()

froz_max=22
# w90data.select_bands(win_min=-np.inf, win_max = 100)

w90data.wannierise(
                init = "amn",
                froz_min=-np.inf,
                froz_max=froz_max,
                print_progress_every=10,
                num_iter=101,
                conv_tol=1e-6,
                mix_ratio_z=1.0,
                sitesym=True,
                )


In [ ]:
from matplotlib import pyplot as plt
from wannierberri.grid import Path
from wannierberri import evaluate_k_path
from wannierberri.system import System_w90
system = System_w90(w90data=w90data)

path = Path(system=system,
            nodes = [[1/2,1/2,1/2], [0,0,0], [1/2,-1/2,0], [3/8, -3/8, 0], [0,0,0]],
            labels = ["L", "$\Gamma$", "X", "K", "$\Gamma$"],
            length=500)

bands_wb = evaluate_k_path(system=system, path=path)

plt.scatter(bands_pw[:,0], bands_pw[:,1], c="black", s=5)



bands_wb.plot_path_fat(path,
                quantity=None,
                # save_file="Fe_bands.pdf",
                Eshift=0,
                Emin=-10, Emax=50,
                iband=None,
                mode="fatband",
                fatfactor=20,
                cut_k=False,
                linecolor="red",
                close_fig=False,
                show_fig=False,
                label=f"WB"
                )

# plt.ylim(-10, 40)
plt.hlines(froz_max, 0, bands_pw[-1,0], linestyles="dashed", label="frozen window", color="black")
plt.legend()
plt.show()